# 모듈 설치

In [ ]:
# google 드라이브 연결
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import sys

print(os.getcwd())

/content


In [ ]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers # >= 4.x.x
!pip install torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 MB 12.9 MB/s eta 0:00:00
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 344.5/344.5 KB 7.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp39-cp39-linux_x86_64.whl size=680540 sha256=a87844ba21dc3ea260687604b24c1e4dbc291ae76915d40d6f3befd04a9f3f7e
  Stored in directory: /root/.cache/pip/wheels/47/17/70/b257bc53879a458c4bfcc900e89271aa8b4f19366a54bd2455
Successfully built gluonnlp
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
#깃허브에서 KoBERT 파일 로드
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-_49daqz8
  Running command git clone --filter=blob:none --quiet 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-_49daqz8
  Resolved https://****@github.com/SKTBrain/KoBERT.git to commit 47a69af87928fc24e20f571fe10c3cc9dd9af9a3
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 KB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.7/54.7 MB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 84.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 69.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 881.9/881.9 MB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 68.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

In [ ]:
#kobert
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

#transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [ ]:
#GPU 사용
device = torch.device("cuda:0")

In [ ]:
#BERT 모델, Vocabulary 불러오기
bertmodel, vocab = get_pytorch_kobert_model()

/content/.cache/kobert_v1.zip[██████████████████████████████████████████████████]
/content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece[██████████████████████████████████████████████████]


# 데이터 획득 및 전처리 

- 데이터 출처 
  - AI 허브 
    - 감성대화말뭉치
    - 감성 및 발화 스타일별 음성 합성 데이터 

In [ ]:
# 전체 데이터 통합본을 데이터셋으로 사용함
import pandas as pd
emotion_df = pd.read_excel('data/감정분류데이터셋.xlsx')

In [ ]:
emotion_df

,sentence,emotion
0,일은 왜 해도 해도 끝이 없을까? 화가 난다.,분노
1,이번 달에 또 급여가 깎였어! 물가는 오르는데 월급만 자꾸 깎이니까 너무 화가 나.,분노
2,회사에 신입이 들어왔는데 말투가 거슬려. 그런 애를 매일 봐야 한다고 생각하니까 스...,분노
3,직장에서 막내라는 이유로 나에게만 온갖 심부름을 시켜. 일도 많은 데 정말 분하고 ...,분노
4,얼마 전 입사한 신입사원이 나를 무시하는 것 같아서 너무 화가 나.,분노
...,...,...
453886,은퇴를 했다가 복귀를 한 경기였다고 해요.,슬픔
453887,온도의 양극화가 되어 있는 식문화가 있습니다.,슬픔
453888,틈이 없다라는 얘기를 가끔씩 듣는데요.,슬픔
453889,"그래서 어, 한 번 만들어볼까 그때 시작됐죠.",슬픔


In [ ]:
emotion_df.shape

(453891, 2)

In [ ]:
emotion_df.describe()

,sentence,emotion
count,453891,453891
unique,163958,6
top,눈물만 흘렸어. 내가 할 수 있는 건 아무것도 없어.,분노
freq,32,82362


In [ ]:
emotion_df.sample(n=10)

,sentence,emotion
189118,이러다 나 혼자가 되는 건 아닌가 안절부절못해.,불안
76268,오늘 기쁜 날이니 아내에게 무언가를 해주고 싶어.,기쁨
354782,정말 취했네.,당황
335497,지금으로서는 방법이 없는 거 같아.,분노
250950,게임에 중독된 것 같은데 쉽게 고쳐지질 않아 초조해.,불안
251126,요즘 무리를 했더니 두드려 맞은 것처럼 아파서 내 생각처럼 일이 안 풀려.,슬픔
265659,그곳 구매자와 벌써 친해졌거든.,기쁨
248472,내가 몸이 불편하다는 이유로 부모님이 날 과잉보호하려고 해서 힘들어.,불안
382765,손끝으로 한번 잡았으면.,슬픔
116597,지금 팀장이랑 달리 전 부장은 일은 못하면서 일을 시키는 것만 좋아하는 사람이었어.,기쁨


In [ ]:
# 데이터 정보 확인
emotion_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 453891 entries, 0 to 453890
Data columns (total 2 columns):
 #   Column    Non-Null Count   Dtype 
---  ------    --------------   ----- 
 0   sentence  453891 non-null  object
 1   emotion   453891 non-null  object
dtypes: object(2)
memory usage: 6.9+ MB


In [ ]:
# emotion 문자열 => 숫자로 변경 
emotion_df.loc[(emotion_df['emotion'] == "불안"), 'emotion'] = 0
emotion_df.loc[(emotion_df['emotion'] == "당황"), 'emotion'] = 1 
emotion_df.loc[(emotion_df['emotion'] == "분노"), 'emotion'] = 2 
emotion_df.loc[(emotion_df['emotion'] == "슬픔"), 'emotion'] = 3  
emotion_df.loc[(emotion_df['emotion'] == "기쁨"), 'emotion'] = 4 
emotion_df.loc[(emotion_df['emotion'] == "상처"), 'emotion'] = 5  

data_list = []
for q, label in zip(emotion_df['sentence'], emotion_df['emotion'])  :
    data = []
    data.append(q)
    data.append(str(label))

    data_list.append(data)

In [ ]:
# 변경 확인 
print(data_list[0])
print(data_list[6000])
print(data_list[12000])
print(data_list[18000])
print(data_list[24000])
print(data_list[30000])
print(data_list[-1])

['일은 왜 해도 해도 끝이 없을까? 화가 난다.', '2']
['최근에 업무가 너무 많이 늘어난 것 같아 힘들어.', '2']
['친구들은 다 취업에 성공했는데 나만 못한 것 같아.', '0']
['친구에게 내가 간암에 걸려서 술을 마실 수 없다고 하자 거짓말하지 말라고 했어.', '5']
['의사가 분명히 수술이 잘 되었다고 했거든. 그런데 삼 주가 되도록 몸을 움직일 수 없어.', '5']
['이번에 주식을 샀는데 주가가 너무 떨어져 손해가 이만저만이 아냐.', '3']
['에이아이 기사 나올 때마다 철렁해요.', '3']


In [ ]:
#train & test 데이터로 나누기
from sklearn.model_selection import train_test_split
                                                         
dataset_train, dataset_test = train_test_split(data_list, test_size=0.2, random_state=0)

In [ ]:
# train, test 데이터 개수 확인
print(len(dataset_train))
print(len(dataset_test))

363112
90779


# 토큰화 

In [ ]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [ ]:
# 파라미터 설정 
max_len = 32
batch_size = 32 
warmup_ratio = 0.1
num_epochs = 5 
max_grad_norm = 1 
log_interval = 200
learning_rate =  5e-5

In [ ]:
dataset_train

[['일단 서운하다고 말이라도 해봐야지. 애인 보러 가기 전에 잠깐이라도 얼굴 비추라고.', '5'],
 ['일단 병원에 가서 치료를 받고 하루에 한 가지씩 나의 긍정적인 면을 찾아 노트에 적어야겠어.', '5'],
 ['무덤을 몇 차례 가면서 친구 이름을 불러도 여전히 외롭고 가슴이 찢어지는 것 같아.', '5'],
 ['내일 신입생 오리엔테이션 가는데 거기서 친구들 못 사귈까 봐 너무 걱정돼.', '5'],
 ['내 말 좀 들어주면 안 되는 건지 모르겠어. 답답하네.', '2'],
 ['그녀의 머리는 색이 바래 이젠 회색빛 아이의 머리카락이었다.', '2'],
 ['나는 그 친구를 믿고 있고 앞으로도 그럴 생각이야.', '4'],
 ['쉬어도 쉰 것 같지가 않고 어디 큰 병 있는 것이 아닌지 모르겠어.', '0'],
 ['아니 어머니는 왜 계속 방에 그냥 막 들어오시는지 모르겠어.', '2'],
 ['언제는 좋다고 하더니 이제 와서 몸에 안 좋다고 하네. 따라한 내가 바보지.', '5'],
 ['자식들을 안 보는 게 좋겠어.', '3'],
 ['소의 위는 무려 네 개나 돼.', '1'],
 ['그럴까? 익숙하지 않아서 어색하네.', '1'],
 ['친구가 멀미 때문에 계속 토하면서 고생을 하고 있는걸 보니 속상하더라고.', '2'],
 ['슬픔을 달래고 피눈물 나는 정성을 다 기울였습니다.', '0'],
 ['동료를 배려하지 않는 태도와 언행이 힘드네.', '0'],
 ['운동을 좀 할까 싶어.', '0'],
 ['밀레니얼 세대가 좋아하는 마지막 특징 뭐가 있을까요?', '3'],
 ['의사의 소견으로는 한쪽에만 있었어도 덜할 거래요.', '2'],
 ['내일은 내일의 해가 뜬다는 말을 좋아하긴 해. 하지만 오늘은 와닿지 않네.', '3'],
 ['영양제를 섭취할지 음식으로 식이요법을 할지 고민했는데 아무래도 영양제가 좀 더 편하겠어.', '4'],
 ['나 이번에도 서류 전형에서 떨어져서 취직 못 했어. 정말 좌절했어.', '2'],
 ['마음대로 취직도

In [ ]:
#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [ ]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=4)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=4)

/usr/local/lib/python3.9/dist-packages/torch/utils/data/dataloader.py:478: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


# 학습 및 모델 저장

In [ ]:
#kobert 학습모델 만들기
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=7,   ##클래스 수 조정##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))[1]
        #bert model returns 'last_hidden_state' and 'pooler_output'

        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [ ]:
#BERT 모델 불러오기
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

#optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

#정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc
    
# train_dataloader

In [ ]:
best_acc=0.0
best_loss=99999999
# 체크포인트 저장 경로 및 이름
ckpt_path="/content/drive/MyDrive/박민이/230313/test/"
ckpt_name=ckpt_path+"saved_model.pt"

In [ ]:
import time
import datetime

In [ ]:
import torch
from torch.utils.tensorboard import SummaryWriter   # 텐서보드에 기록을 위해
writer = SummaryWriter()
import datetime

time_stamp = str(datetime.datetime.now().timestamp())
log_dir = "logs/kobert" + time_stamp
writer = SummaryWriter(log_dir)

%load_ext tensorboard

In [ ]:
#kobert 모델 학습시키기
# 시간 확인 추가
start = time.time()
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    # train
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        writer.add_scalar("Loss/train", loss.data.cpu().numpy(), e+1)
        writer.add_scalar("Accuracy/train", train_acc / (batch_id+1), e+1)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    
    # test
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        with torch.no_grad():
            out = model(token_ids, valid_length, segment_ids)
        test_loss=loss_fn(out,label)
        test_acc += calc_accuracy(out, label)
        writer.add_scalar("Loss/test", test_loss.data.cpu().numpy(), e+1)
        writer.add_scalar("Accuracy/test", test_acc / (batch_id+1), e+1)
    print("epoch {} test acc {} test loss {}".format(e+1, test_acc / (batch_id+1),test_loss.data.cpu().numpy()))

    # 모델 저장
    if test_acc>best_acc and test_loss.data.cpu().numpy()<best_loss:
      torch.save({'epoch':e+1,
                  'model_state_dict':model.state_dict(),
                  'optimizer_state_dict':optimizer.state_dict(),
                  'loss':test_loss.data.cpu().numpy()},
                 ckpt_name)
      best_loss=test_loss.data.cpu().numpy()
      bset_acc=test_acc
      
      print('current best model saved')
end = time.time()
print(str(datetime.timedelta(seconds=(end - start))))
    

<ipython-input-32-5b122e147dbf>:8: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(train_dataloader)):


  0%|          | 0/11348 [00:00<?, ?it/s]

/usr/local/lib/python3.9/dist-packages/torch/utils/data/dataloader.py:478: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


epoch 1 batch id 1 loss 2.0483481884002686 train acc 0.125
epoch 1 batch id 201 loss 1.9418460130691528 train acc 0.16635572139303484
epoch 1 batch id 401 loss 1.815184235572815 train acc 0.17495324189526185
epoch 1 batch id 601 loss 1.7269572019577026 train acc 0.17782861896838603
epoch 1 batch id 801 loss 1.7231475114822388 train acc 0.1972534332084894
epoch 1 batch id 1001 loss 1.5227525234222412 train acc 0.2209040959040959
epoch 1 batch id 1201 loss 1.4727981090545654 train acc 0.24378122398001664
epoch 1 batch id 1401 loss 1.6521515846252441 train acc 0.26075124910778014
epoch 1 batch id 1601 loss 1.5433377027511597 train acc 0.2771509993753904
epoch 1 batch id 1801 loss 1.466434359550476 train acc 0.28924902831760135
epoch 1 batch id 2001 loss 1.8376184701919556 train acc 0.2989598950524738
epoch 1 batch id 2201 loss 1.3956401348114014 train acc 0.30839675147660156
epoch 1 batch id 2401 loss 1.2875220775604248 train acc 0.3166389004581424
epoch 1 batch id 2601 loss 1.52645254135

<ipython-input-32-5b122e147dbf>:29: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_dataloader)):


...

epoch 4 test acc 0.8050256041201581 test loss 0.3500104546546936
current best model saved


  0%|          | 0/11348 [00:00<?, ?it/s]

epoch 5 batch id 1 loss 0.5513920187950134 train acc 0.8125
epoch 5 batch id 201 loss 0.6872493028640747 train acc 0.851679104477612
epoch 5 batch id 401 loss 0.7702990770339966 train acc 0.8531795511221946
epoch 5 batch id 601 loss 0.4260767102241516 train acc 0.8543573211314476
epoch 5 batch id 801 loss 0.33492758870124817 train acc 0.8558832709113608
epoch 5 batch id 1001 loss 0.2856930196285248 train acc 0.8578296703296703
epoch 5 batch id 1201 loss 0.7770000100135803 train acc 0.856838051623647
epoch 5 batch id 1401 loss 0.44155246019363403 train acc 0.8567317987152034
epoch 5 batch id 1601 loss 0.5535303354263306 train acc 0.8568472829481574
epoch 5 batch id 1801 loss 0.4191121459007263 train acc 0.8562777623542477
epoch 5 batch id 2001 loss 0.5759694576263428 train acc 0.8568528235882059
epoch 5 batch id 2201 loss 0.2237720936536789 train acc 0.8577635165833712
epoch 5 batch id 2401 loss 0.369509756565094 train acc 0.8586916909620991
epoch 5 batch id 2601 loss 0.4608491659164428

  0%|          | 0/2837 [00:00<?, ?it/s]

epoch 5 test acc 0.8223304155406728 test loss 0.342564195394516
current best model saved
3:39:15.879717


In [ ]:
writer.flush()
writer.close()

In [ ]:
# 그래프 나타남
%tensorboard --logdir {log_dir}